# Task1-模型微調

In [ ]:
!pip install  datasets evaluate jiwer librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/aicup')
import json
import torch
import random
import librosa
import zipfile
import evaluate
import numpy as np
import pandas as pd
from datasets import Dataset, Audio, load_dataset, Features, Value
from aicup import (DataCollatorSpeechSeq2SeqWithPadding,
      transcribe_with_timestamps,
      collate_batch_with_prompt_template,
      generate_annotated_audio_transcribe_parallel,OpenDeidBatchSampler)
from transformers import (
    WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor,
    WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
)

In [ ]:
import os, zipfile

In [ ]:
zip_path = "/content/drive/MyDrive/Final_Dataset.zip"
extract_dir = "/content/audio_dataset"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)

audio_files = []
for root, dirs, files in os.walk(extract_dir):
  for file in files:
    if file.lower().endswith('.wav'):
      audio_files.append(os.path.join(root, file))

print(f"found {len(audio_files)} audio files")
# for path in audio_files:
#   print(path)

found 2314 audio files


In [ ]:
def set_torch_seed(seed=0):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benckmark = False
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_torch_seed()

In [ ]:
t1_train_audio_folder = "/content/audio_dataset/audio"
t1_train_transcription_files = [
    "/content/audio_dataset/task1_answer_01.txt",
    "/content/audio_dataset/task1_answer_02.txt",
    "/content/audio_dataset/task1_answer_03.txt",
]

transcripts, dataset_list = {}, []

for txt_path in t1_train_transcription_files:
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split("\t", 1)
            if len(parts) == 2:
                filename, transcript = parts
                transcripts[filename] = transcript

for file in sorted(os.listdir(t1_train_audio_folder)):
  if file.endswith(".wav") and file.split(".")[0] in transcripts:
    try:
      file_path = os.path.join(t1_train_audio_folder, file)
      audio_array, sr = librosa.load(file_path, sr=16000)
      dataset_list.append({"audio":
                 {
                  "path":file_path,
                  "array":audio_array,
                  "sampling_rate":sr
                 },
                 "sentence": transcripts[file.split(".")[0]]})
    except Exception as e:
      print(e)
      print(f"Can't read {file_path}:{e}")

dataset = Dataset.from_pandas(pd.DataFrame(dataset_list))

In [ ]:
dataset[0]['audio']['array'][:20]

[8.841743692755699e-07,
 -4.072557203471661e-06,
 -2.194545231759548e-06,
 8.789938874542713e-07,
 -4.918547347187996e-07,
 2.3690517991781235e-07,
 7.351627573370934e-08,
 -1.140229869633913e-06,
 -6.318034138530493e-06,
 -3.266148269176483e-06,
 6.970658432692289e-07,
 -5.945767043158412e-07,
 7.412454579025507e-07,
 -1.0783551260828972e-06,
 1.8741702660918236e-06,
 -6.328918971121311e-06,
 -6.2480103224515915e-06,
 1.7824932001531124e-06,
 -9.547220543026924e-07,
 5.385954864323139e-07]

In [ ]:
print("audio sample:",len(dataset[0]['audio']['array']))
print("audio duration:",len(dataset[0]['audio']['array'])/16000)

audio sample: 375968
audio duration: 23.498


In [ ]:
split_ratio = 0.8
train_size = int(len(dataset) * split_ratio)
dataset = dataset.train_test_split(train_size=train_size,
     test_size=len(dataset) - train_size, shuffle=True, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(f"Train: {len(train_dataset)} samples, Test: {len(test_dataset)} samples")

Train: 1851 samples, Test: 463 samples


In [ ]:
test_dataset[0]

{'audio': {'array': [-0.0005388393765315413,
   -0.0007346562924794853,
   -0.0005723579088225961,
   -0.0004939547507092357,
   -0.00032234026002697647,
   -0.0008772700093686581,
   -0.0005567760090343654,
   -0.0005329450359568,
   -0.0003835495444945991,
   -0.0005691087571904063,
   -0.0007211666088551283,
   -0.0005897241644561291,
   -0.0004144668346270919,
   -0.00047083699610084295,
   -0.0006148978136479855,
   -0.00046591059071943164,
   -0.0005906636361032724,
   -0.0002652513503562659,
   -9.369554027216509e-05,
   0.00011392820306355134,
   -0.00026614422677084804,
   5.785219400422648e-06,
   -0.00016280806448776275,
   -4.390891263028607e-06,
   -0.00026119122048839927,
   -0.0006246286211535335,
   -5.775643512606621e-05,
   -0.00018745698616839945,
   -0.00017690766253508627,
   -0.0003655084001366049,
   -0.00020572368521243334,
   1.9791676095337607e-05,
   0.00015657258336432278,
   -1.910261380544398e-05,
   -1.3618240700452588e-05,
   -0.00012710540613625199,
   

In [ ]:
model_name = "Na0s/Medical-Whisper-Large-v3"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.generation_config.language = None
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

In [ ]:
# print(processor.feature_extractor(test_dataset[2]['audio']['array'],
#     sampling_rate=test_dataset[2]['audio']["sampling_rate"])['input_features'])
print(tokenizer(test_dataset[2]['sentence']).input_ids)

[50258, 50364, 11645, 452, 3152, 27670, 286, 1415, 281, 978, 293, 1780, 562, 436, 1293, 34329, 2428, 13, 286, 362, 2825, 281, 452, 27851, 466, 300, 886, 359, 264, 12308, 1507, 13, 1144, 291, 420, 362, 291, 1562, 632, 257, 12308, 1393, 30, 286, 1143, 281, 562, 286, 390, 257, 21440, 13, 286, 576, 747, 257, 3840, 295, 23871, 420, 445, 483, 666, 452, 1225, 913, 247, 82, 1032, 293, 1261, 309, 322, 293, 352, 281, 2817, 294, 264, 5395, 12, 1010, 14400, 13, 663, 390, 562, 286, 390, 294, 16195, 1090, 293, 1090, 1395, 13, 50257]


In [ ]:
def preprocess_dataset(batch):

  audio = batch["audio"]
  batch["input_features"] = processor.feature_extractor(audio["array"],
    sampling_rate=audio["sampling_rate"]).input_features[0]
  batch["labels"] = tokenizer(batch["sentence"]).input_ids

  return batch

train_dataset = train_dataset.map(preprocess_dataset, remove_columns=dataset.column_names["train"]) #,num_proc=4
test_dataset = test_dataset.map(preprocess_dataset, remove_columns=dataset.column_names["test"])

Map:   0%|          | 0/1851 [00:00<?, ? examples/s]

Map:   0%|          | 0/463 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
d = [test_dataset[i] for i in range(2)]
batch = data_collator(d)
print(batch['input_features'])
print(batch['labels'])

tensor([[[-0.4361, -0.0707, -0.6097,  ..., -0.9930, -0.9930, -0.9930],
         [-0.3385,  0.0269, -0.5122,  ..., -0.9930, -0.9930, -0.9930],
         [-0.0373, -0.3017, -0.1486,  ..., -0.9930, -0.9930, -0.9930],
         ...,
         [-0.7171, -0.7583, -0.7631,  ..., -0.9930, -0.9930, -0.9930],
         [-0.7378, -0.7826, -0.8856,  ..., -0.9930, -0.9930, -0.9930],
         [-0.9350, -0.9930, -0.9930,  ..., -0.9930, -0.9930, -0.9930]],

        [[-0.8439, -0.8439, -0.8439,  ..., -0.8439, -0.8439, -0.8439],
         [-0.8439, -0.8439, -0.8439,  ..., -0.8439, -0.8439, -0.8439],
         [-0.8439, -0.8439, -0.8439,  ..., -0.8439, -0.8439, -0.8439],
         ...,
         [-0.8439, -0.8439, -0.8439,  ..., -0.8439, -0.8439, -0.8439],
         [-0.8439, -0.8439, -0.8439,  ..., -0.8439, -0.8439, -0.8439],
         [-0.8439, -0.8439, -0.8439,  ..., -0.8439, -0.8439, -0.8439]]])
tensor([[50364,   511, 20932,  1890,  1081,   294,   264,   805,    14,    24,
         12342,   414, 41576, 16346, 

In [ ]:
from jiwer import mer
from transformers.models.whisper.english_normalizer import BasicTextNormalizer


normalizer = BasicTextNormalizer()

def mixed_tokenizer(text):
  text = normalizer(text.strip())
  tokens = []
  temp_token = ""
  # print(text)
  for char in text:
    if '\u4e00' <= char <= '\u9fff':
      if temp_token:
          tokens.append(temp_token)
          temp_token = ""
      tokens.append(char)
    elif char.isspace():
      if temp_token:
        tokens.append(temp_token)
        temp_token = ""
    else:
      temp_token += char
  if temp_token:
    tokens.append(temp_token)
  return tokens

def compute_metrics(eval_pred):
  predictions, labels = eval_pred

  decoded_preds = processor.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)

  decoded_preds = [normalizer(pred.strip()) for pred in decoded_preds]
  decoded_labels = [normalizer(label.strip()) for label in decoded_labels]

  paired = [
      (ref, hyp) for ref, hyp in zip(decoded_labels, decoded_preds)
      if ref.strip() != "" and hyp.strip() != ""
  ]

  filtered_labels, filtered_preds = zip(*paired) if paired else ([], [])
  if len(filtered_labels) == 0:
    return {"mer": 1.0}

  ref_tokens = [mixed_tokenizer(t) for t in filtered_labels]
  pred_tokens = [mixed_tokenizer(t) for t in filtered_preds]
  # print(ref_tokens)
  # print(pred_tokens)
  ref_strs = [" ".join(tokens) for tokens in ref_tokens]
  pred_strs = [" ".join(tokens) for tokens in pred_tokens]

  try:
    score = mer(ref_strs, pred_strs)
  except Exception as e:
    print("Error during MER computation:", e)
    score = 1.0

  return {"mer": score}

In [ ]:
for param in model.model.encoder.parameters():
  param.requires_grad = False

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Trainable parameters: {trainable_params:,}")
print(f"Total parameters:     {total_params:,}")
print(f"Trainable ratio:      {trainable_params / total_params:.2%}")

Trainable parameters: 906,521,600
Total parameters:     1,543,490,560
Trainable ratio:      58.73%


In [ ]:
training_args = Seq2SeqTrainingArguments(
  output_dir="/content/models/Na0s/Medical-Whisper-Large-v3",
  report_to="none",
  num_train_epochs=5,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  eval_strategy="epoch",
  save_strategy="epoch",
  predict_with_generate=True,
  logging_dir="/content/models/Na0s/Medical-Whisper-Large-v3/logs",
  logging_steps=1,
  fp16=True,
  learning_rate=3e-6,
  warmup_ratio=0.1,
  gradient_accumulation_steps=4,
  dataloader_num_workers=4,
)

trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=test_dataset,
  tokenizer=processor.tokenizer,
  data_collator=data_collator,
  compute_metrics=None
)

<ipython-input-18-9571a4f3fb5c>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.505500,0.487555
2,0.337600,0.368348
3,0.291300,0.355836
4,0.237100,0.352932
5,0.222200,0.353624


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=290, training_loss=0.4337510092505093, metrics={'train_runtime': 824.2938, 'train_samples_per_second': 11.228, 'train_steps_per_second': 0.352, 'total_flos': 3.14438468272128e+19, 'train_loss': 0.4337510092505093, 'epoch': 5.0})

In [ ]:
save_directory = "/content/drive/MyDrive/aicup/models/Na0s-Medical-Whisper-Large-v3"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
processor.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")

Model saved to /content/drive/MyDrive/aicup/models/Na0s-Medical-Whisper-Large-v3


# Output result-whiser句子級+whisperX字詞級時間軸

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re, math, os, zipfile, json, torch, librosa
import pandas as pd
from datasets import Dataset
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
)

In [ ]:
zip_path = "/content/drive/MyDrive/Private_dataset.zip"
extract_dir = "/content/audio_dataset"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)

In [ ]:
extract_dir = "/content/audio_dataset"
audio_files = []
for root, dirs, files in os.walk(extract_dir):
  for file in files:
    if file.lower().endswith('.wav'):
      audio_files.append(os.path.join(root, file))

print(f"found {len(audio_files)} audio files")
# for path in audio_files:
#   print(path)

found 710 audio files


In [ ]:
valid_list = []
audio_folder = "/content/audio_dataset/private"
for fname in sorted(os.listdir(audio_folder)):
    if not fname.endswith(".wav"):
        continue
    path = os.path.join(audio_folder, fname)
    try:
        arr, sr = librosa.load(path, sr=16000)
    except Exception as e:
        print(f"Can't read {path}: {e}")
        continue
    valid_list.append({
        "audio": {"path": path, "array": arr, "sampling_rate": sr},
        "sentence": ""
    })
valid_dataset = Dataset.from_pandas(pd.DataFrame(valid_list))

In [ ]:
model_name = "/content/drive/MyDrive/aicup/models/Na0s-Medical-Whisper-Large-v3"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer         = WhisperTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
processor = WhisperProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.generation_config.forced_decoder_ids = None
model.generation_config.task = "transcribe"
model.generation_config.language = "en"
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).eval()

def transcribe_with_timestamps(item, model, processor):
    arr = item["audio"]["array"]
    sr  = item["audio"]["sampling_rate"]
    inputs = processor.feature_extractor(
        arr,
        sampling_rate=sr,
        padding="longest",
        return_tensors="pt",
        return_attention_mask=True
    ).to(device)
    with torch.no_grad():
        gen = model.generate(
            inputs.input_features,
            attention_mask=inputs.attention_mask,
            task="transcribe",
            language="en",
            return_dict_in_generate=True,
            return_timestamps=True,
        )
    seq = gen["sequences"][0]
    text = processor.decode(seq, skip_special_tokens=True).strip()
    segments = []
    for seg in gen["segments"][0]:
        word  = processor.decode(seg["tokens"], skip_special_tokens=True).strip()
        start = seg["start"].item()
        end   = seg["end"].item()
        segments.append({
            "word":  word,
            "start": round(start, 2),
            "end":   round(end,   2),})
    return {"text": text, "segments": segments}

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
out_txt  = "/content/drive/MyDrive/task1_answer.txt"
out_json = "/content/task1_answer_timestamps.json"
mapping = {}

with open(out_txt, "w", encoding="utf-8") as f_txt:
    for item in valid_dataset:
        res = transcribe_with_timestamps(item)
        key = os.path.splitext(os.path.basename(item["audio"]["path"]))[0]
        mapping[key] = res
        f_txt.write(f"{key}\t{res['text']}\n")

with open(out_json, "w", encoding="utf-8") as f_json:
    json.dump(mapping, f_json, ensure_ascii=False, indent=2)

In [ ]:
!pip install whisperx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 134.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import importlib
import whisperx
import torch

alignment = importlib.import_module("whisperx.alignment")
def _patched_get_wildcard_emission(frame_emission, tokens, blank_id):
    if not isinstance(tokens, torch.Tensor):
        tokens = torch.tensor(tokens, device=frame_emission.device)
    tokens = tokens.to(torch.long)
    regular_scores = frame_emission[tokens.clamp(min=0)]
    mask = tokens == -1
    if mask.any():
        wild_scores, _ = frame_emission.max(dim=-1, keepdim=True)
        wild_broadcast = wild_scores.expand(*regular_scores.shape)
        regular_scores[mask] = wild_broadcast[mask]
    return regular_scores
alignment.get_wildcard_emission = _patched_get_wildcard_emission

json_path      = "/content/task1_answer_timestamps.json"
audio_root_dir = "/content/audio_dataset/private"
outfile        = "/content/drive/MyDrive/task1_word_timestamps.json"

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

file2segments = {}
for fid, rec in data.items():
    wav = os.path.join(audio_root_dir, f"{fid}.wav")
    if not os.path.isfile(wav):
        print(f"⚠ 找不到音檔 {wav}，跳過")
        continue
    file2segments[wav] = [
        {"start": seg["start"], "end": seg["end"], "text": seg["word"]}
        for seg in rec["segments"]
    ]


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
align_en, meta_en = whisperx.load_align_model(language_code="en", device=device)
align_zh, meta_zh = whisperx.load_align_model(language_code="zh", device=device)

def is_chinese(txt: str) -> bool:
    return any('\u4e00' <= ch <= '\u9fff' for ch in txt)

word_level_dict = {}
for wav_path, segs in file2segments.items():
    fid = os.path.splitext(os.path.basename(wav_path))[0]
    print(f"Aligning {fid}…")
    audio = whisperx.load_audio(wav_path)

    en_segs = [s for s in segs if not is_chinese(s["text"])]
    zh_segs = [s for s in segs if is_chinese(s["text"])]

    aligned_words = []
    if en_segs:
        out_en = whisperx.align(
            en_segs, align_en, meta_en,
            audio, device,
            return_char_alignments=False
        )
        for w in out_en.get("word_segments", []):
            txt = w.get("word") or w.get("text") or w.get("label")
            aligned_words.append({"word": txt, "start": w["start"], "end": w["end"]})
    if zh_segs:
        out_zh = whisperx.align(
            zh_segs, align_zh, meta_zh,
            audio, device,
            return_char_alignments=False
        )
        for w in out_zh.get("word_segments", []):
            txt = w.get("word") or w.get("text") or w.get("label")
            aligned_words.append({"word": txt, "start": w["start"], "end": w["end"]})

    word_level_dict[fid] = sorted(aligned_words, key=lambda x: x["start"])

with open(outfile, 'w', encoding='utf-8') as f:
    json.dump(word_level_dict, f, ensure_ascii=False, indent=2)

print(f"已輸出字詞級時間軸到：{outfile}")

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 364MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/44.4k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

Aligning 60014…
Aligning 60015…
Aligning 60018…
Aligning 60022…
Aligning 60049…
Aligning 60079…
Aligning 60084…
Aligning 60102…
Aligning 60147…
Aligning 60167…
Aligning 60180…
Aligning 60184…
Aligning 60197…
Aligning 60226…
Aligning 60239…
Aligning 60243…
Aligning 60245…
Aligning 60253…
Aligning 60267…
Aligning 60275…
Aligning 60286…
Aligning 60290…
Aligning 60330…
Aligning 60349…
Aligning 60362…
Aligning 60365…
Aligning 60373…
Aligning 60376…
Aligning 60380…
Aligning 60387…
Aligning 60394…
Aligning 60420…
Aligning 60466…
Aligning 60500…
Aligning 60514…
Aligning 60533…
Aligning 60535…
Aligning 60559…
Aligning 60574…
Aligning 60581…
Aligning 60598…
Aligning 60617…
Aligning 60626…
Aligning 60633…
Aligning 60697…
Aligning 60699…
Aligning 60703…
Aligning 60730…
Aligning 60732…
Aligning 60740…
Aligning 60741…
Aligning 60769…
Aligning 60779…
Aligning 60786…
Aligning 60791…
Aligning 60802…
Aligning 60814…
Aligning 60830…
Aligning 60832…
Aligning 60884…
Aligning 60905…
Aligning 60914…
Aligning

# Task2-前置

In [ ]:
import os, sys
sys.path.append('/content/drive/MyDrive/aicup')
from aicup import generate_annotated_audio_transcribe_parallel,OpenDeidBatchSampler
task2_train_answer = r"/content/drive/MyDrive/Final_Training_Dataset/task2_answer_03.txt"
task2_train_transcribe = r"/content/drive/MyDrive/Final_Training_Dataset/task1_answer_03.txt"
task2_train_data = '/content/drive/MyDrive/Final_Training_Dataset/task2_train_03.tsv'
generate_annotated_audio_transcribe_parallel(task2_train_answer, task2_train_transcribe, task2_train_data, num_processes=4)

process annotation file...
annotation file done
processing each medical file
2561	Yeah. But I've been dreading talking about it and it wasn't so bad, you know. Just about what we were going to do as far as selling, you know. And I said I was going to do something. Oh, see if they have an investment modification program. Maybe I'll call them today or tomorrow. This is the bank? Yeah. Just to buy some time, you know. My condo litigation is just I'm on all these emails that I can't even read them.	DATE:today\nDATE:tomorrow

775
All medical file done
write out to tsv format...
tsv format dataset done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade bitsandbytes transformers==4.50.0 accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 76.1/76.1 MB 193.5 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
import os, sys, re, torch, random
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, load_metric
sys.path.append('/content/drive/MyDrive/aicup')
from aicup import (DataCollatorSpeechSeq2SeqWithPadding,
      transcribe_with_timestamps,
      collate_batch_with_prompt_template,
      generate_annotated_audio_transcribe_parallel,OpenDeidBatchSampler)
from transformers import (
    WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, GenerationConfig,
    WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, pipeline,
    AutoConfig, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback
)
def set_torch_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benckmark = False
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_torch_seed()

In [ ]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=7005fc6dac8f801060ddaf38f22def31a4bd3f5225855538d738de56868ce4b2
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score
from evaluate import load

# Token-Classification+延伸標籤微調

In [ ]:
files = [
    '/content/drive/MyDrive/Final_Training_Dataset/task2_train_01.tsv',
    '/content/drive/MyDrive/Final_Training_Dataset/task2_train_02.tsv',
    '/content/drive/MyDrive/Final_Training_Dataset/task2_train_03.tsv',
    '/content/drive/MyDrive/Final_Training_Dataset/task2_train_04.tsv',
    '/content/drive/MyDrive/Final_Training_Dataset/task2_train_05.tsv',
    '/content/drive/MyDrive/Final_Training_Dataset/task2_train_06.tsv',
]

df_list = [pd.read_csv(f, sep='\t', names=['fid','content','label'], dtype=str, keep_default_na=False)
           for f in files]
df_all = pd.concat(df_list, ignore_index=True)
raw_ds = Dataset.from_pandas(df_all, preserve_index=False)
dataset = DatasetDict({'train': raw_ds})
dataset

DatasetDict({
    train: Dataset({
        features: ['fid', 'content', 'label'],
        num_rows: 9807
    })
})

In [ ]:
split = dataset['train'].train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    'train':   split['train'],
    'validation': split['test']
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['fid', 'content', 'label'],
        num_rows: 8826
    })
    validation: Dataset({
        features: ['fid', 'content', 'label'],
        num_rows: 981
    })
})


In [ ]:
from collections import Counter

ctr = Counter()

for i in dataset['train']:
  phi_labelwvalue = i['label'].split("\\n")
  phi_label = [j.split(":")[0] for j in phi_labelwvalue]
  ctr.update(phi_label)

print(ctr)

Counter({'DATE': 18795, 'DOCTOR': 8607, 'DURATION': 7848, 'PATIENT': 5801, 'TIME': 4509, 'PERSONALNAME': 4028, 'ID_NUMBER': 3761, 'FAMILYNAME': 2806, 'CITY': 1905, 'HOSPITAL': 1830, 'MEDICAL_RECORD_NUMBER': 1669, 'STATE': 1636, 'DEPARTMENT': 1600, 'STREET': 1459, 'ZIP': 1356, 'SET': 1153, 'AGE': 900, 'PHI': 785, 'PROFESSION': 534, 'ORGANIZATION': 516, 'LOCATION-OTHER': 343, 'COUNTRY': 214, 'COUNTY': 48, 'ROOM': 46, 'PHONE': 26, 'DISTRICT': 21, 'URL': 17})


In [ ]:
phi_list = ['PATIENT', 'DOCTOR', 'USERNAME', 'PERSONALNAME', 'FAMILYNAME', 'PROFESSION', 'ROOM', 'DEPARTMENT',
            'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY', 'STATE', 'COUNTRY', 'COUNTY', 'ZIP', 'LOCATION-OTHER',
            'DISTRICT', 'AGE', 'DATE', 'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDRESS', 'OTHER',
            'SOCIAL_SECURITY_NUMBER', 'MEDICAL_RECORD_NUMBER', 'HEALTH_PLAN_NUMBER', 'ACCOUNT_NUMBER', 'LICENSE_NUMBER', 'VEHICLE_ID',
            'DEVICE_ID', 'BIOMETRIC_ID', 'ID_NUMBER']
labels = ["O"] + [f"{p}-{lbl}" for lbl in phi_list for p in ("B","I")]
LABEL2ID = {l:i for i,l in enumerate(labels)}
ID2LABEL = {i:l for l,i in LABEL2ID.items()}
print(LABEL2ID)

{'O': 0, 'B-PATIENT': 1, 'I-PATIENT': 2, 'B-DOCTOR': 3, 'I-DOCTOR': 4, 'B-USERNAME': 5, 'I-USERNAME': 6, 'B-PERSONALNAME': 7, 'I-PERSONALNAME': 8, 'B-FAMILYNAME': 9, 'I-FAMILYNAME': 10, 'B-PROFESSION': 11, 'I-PROFESSION': 12, 'B-ROOM': 13, 'I-ROOM': 14, 'B-DEPARTMENT': 15, 'I-DEPARTMENT': 16, 'B-HOSPITAL': 17, 'I-HOSPITAL': 18, 'B-ORGANIZATION': 19, 'I-ORGANIZATION': 20, 'B-STREET': 21, 'I-STREET': 22, 'B-CITY': 23, 'I-CITY': 24, 'B-STATE': 25, 'I-STATE': 26, 'B-COUNTRY': 27, 'I-COUNTRY': 28, 'B-COUNTY': 29, 'I-COUNTY': 30, 'B-ZIP': 31, 'I-ZIP': 32, 'B-LOCATION-OTHER': 33, 'I-LOCATION-OTHER': 34, 'B-DISTRICT': 35, 'I-DISTRICT': 36, 'B-AGE': 37, 'I-AGE': 38, 'B-DATE': 39, 'I-DATE': 40, 'B-TIME': 41, 'I-TIME': 42, 'B-DURATION': 43, 'I-DURATION': 44, 'B-SET': 45, 'I-SET': 46, 'B-PHONE': 47, 'I-PHONE': 48, 'B-FAX': 49, 'I-FAX': 50, 'B-EMAIL': 51, 'I-EMAIL': 52, 'B-URL': 53, 'I-URL': 54, 'B-IPADDRESS': 55, 'I-IPADDRESS': 56, 'B-OTHER': 57, 'I-OTHER': 58, 'B-SOCIAL_SECURITY_NUMBER': 59, 'I-S

In [ ]:
extended_to_base = {
    "PATIENT_NAME": "PATIENT", "SUBJECT": "PATIENT",
    "DOCTOR_NAME": "DOCTOR", "PHYSICIAN": "DOCTOR", "DR": "DOCTOR",
    "LOGIN_NAME": "USERNAME", "USER_ID": "USERNAME",
    "EMAIL_ADDRESS": "EMAIL", "MAIL": "EMAIL",
    "IP_ADDR": "IPADDRESS", "IP_ADDRESS": "IPADDRESS", "IPADDR": "IPADDRESS",
    "SSN": "SOCIAL_SECURITY_NUMBER", "MRN": "MEDICAL_RECORD_NUMBER",
    "HPN": "HEALTH_PLAN_NUMBER", "ACCT_NO": "ACCOUNT_NUMBER", "LIC_NO": "LICENSE_NUMBER",
    "VIN": "VEHICLE_ID", "DEV_ID": "DEVICE_ID",
    "FINGERPRINT_ID": "BIOMETRIC_ID", "FACE_ID": "BIOMETRIC_ID",
    "ID_NO": "ID_NUMBER", "PASSPORT_NUMBER": "ID_NUMBER",

    "RELDATE": "DATE", "WEEKDAY": "DATE", "MONTH": "DATE",
    "TIME_FULL": "TIME", "TIME_SHORT": "TIME", "POD": "TIME", "TIME_ZONE": "TIME",
    "DURATION": "DURATION", "SET": "SET",
    "PHONE": "PHONE", "FAX": "FAX",
    "URL": "URL",
    "PERCENT": "SET", "MONEY": "OTHER", "DECIMAL": "OTHER", "FRACTION": "OTHER",
    "RANGE_DATE": "DATE", "RANGE_TIME": "TIME",
    **{lbl: lbl for lbl in phi_list}
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("michiyasunaga/BioLinkBERT-large", use_fast=True)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized = tokenizer(
        examples["content"],
        truncation=True,
        padding=False,
        max_length=512,
        return_offsets_mapping=True
    )
    decoded_labels = [lab.encode("utf-8").decode("unicode_escape") for lab in examples["label"]]

    all_labels = []
    for i, offsets in enumerate(tokenized["offset_mapping"]):
        text_lower = examples["content"][i].lower()
        labels = ["O"] * len(offsets)
        valid_idxs = [j for j,(s,e) in enumerate(offsets) if not (s==e==0)]

        for line in decoded_labels[i].split("\n"):
            if ":" not in line: continue
            key, val = [x.strip() for x in line.split(":",1)]
            key = key.upper()
            key = extended_to_base.get(key, key)
            if key == "PHI" and val.lower() == "null":
              continue
            val_clean = val.strip().strip('.,;:\"\'').lower()
            spans = [m.span() for m in re.finditer(re.escape(val_clean), text_lower)]
            if not spans:
                idx = text_lower.find(val_clean)
                if idx != -1:
                    spans = [(idx, idx+len(val_clean))]
                else:
                    trimmed = re.sub(r"[^0-9a-z]+", " ", val_clean).strip()
                    idx = text_lower.find(trimmed)
                    if idx != -1:
                        spans = [(idx, idx+len(trimmed))]

            for (st, ed) in spans:
                toks = [j for j in valid_idxs if not (offsets[j][1] <= st or offsets[j][0] >= ed)]
                if not toks: continue
                first = True
                for j in toks:
                    labels[j] = f"{'B' if first else 'I'}-{key}"
                    first = False
        all_labels.append(labels)

    tokenized.pop("offset_mapping")
    tokenized["labels_str"] = all_labels
    tokenized["labels"] = [[LABEL2ID.get(l, 0) for l in lab_seq] for lab_seq in all_labels]
    return tokenized

tokenized_ds = dataset.map(
     tokenize_and_align_labels,
     batched=True,
     remove_columns=["fid","content","label"])

for ex in tokenized_ds["train"].select(range(5)):
    print(ex["labels_str"])

Map:   0%|          | 0/8826 [00:00<?, ? examples/s]

Map:   0%|          | 0/981 [00:00<?, ? examples/s]

['O', 'O', 'O', 'O', 'O', 'B-CITY', 'I-CITY', 'I-CITY', 'I-CITY', 'I-CITY', 'I-CITY', 'O', 'O', 'O', 'O', 'O', 'B-PATIENT', 'I-PATIENT', 'I-PATIENT', 'I-PATIENT', 'I-PATIENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'O', 'O', 'O', 'O', 'O', 'B-TIME', 'I-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'B-PATIENT', 'I-PATIENT', 'O', 'O', 'O', 'O', 'O', 'B-STREET', 'I-STREET', 'O', 'O', 'O', 'O', 'O', 'B-DOCTOR', 'I-DOCTOR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'I-ID_NUMBER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERSONALNAME', 'I-PERSONALNAME', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'O', 'B-PATIENT', 'I-PATIENT', 'I-PATIENT', 'I-PATIENT', 'I-PATIENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-

In [ ]:
import itertools
from collections import Counter
cnt = Counter(itertools.chain.from_iterable(tokenized_ds["train"]["labels"]))
print("#token =", sum(cnt.values()))
print("Top labels (id → 次數):", cnt.most_common(10))


#token = 760851
Top labels (id → 次數): [(0, 564954), (40, 29304), (39, 18132), (76, 15110), (4, 14480), (2, 10420), (44, 9562), (62, 8619), (3, 7811), (43, 7693)]


In [ ]:
for i in range(5):
    print("---- 範例", i, "----")
    content = dataset["train"][i]["content"]
    raw_lab = dataset["train"][i]["label"]

    print("原文 content:", content)
    print("原始 label :", raw_lab)

    decoded_labels = raw_lab.split("\n")
    print("decoded_labels:", decoded_labels)
    print()


---- 範例 0 ----
原文 content: The city mentioned was Orbost, Tasmania. The patient mentioned was Oliver Antonio. The id_number mentioned was 83Z179706W. The time mentioned was 10:15 AM. The patient mentioned was Jolin. The street mentioned was Main Line. The doctor mentioned was Porter. The time mentioned was 兩點半. The id_number mentioned was 96C259313O. The personalname mentioned was James.
原始 label : CITY:Orbost, Tasmania\nPATIENT:Oliver Antonio\nID_NUMBER:83Z179706W\nTIME:10:15 AM\nPATIENT:Jolin\nSTREET:Main Line\nDOCTOR:Porter\nTIME:兩點半\nID_NUMBER:96C259313O\nPERSONALNAME:James
decoded_labels: ['CITY:Orbost, Tasmania\\nPATIENT:Oliver Antonio\\nID_NUMBER:83Z179706W\\nTIME:10:15 AM\\nPATIENT:Jolin\\nSTREET:Main Line\\nDOCTOR:Porter\\nTIME:兩點半\\nID_NUMBER:96C259313O\\nPERSONALNAME:James']

---- 範例 1 ----
原文 content: The patient mentioned was 江朝正. The date mentioned was first week. The patient mentioned was Rodolfo Ruman. The duration mentioned was 三週. The date mentioned was December 28, 2

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "michiyasunaga/BioLinkBERT-large",
    num_labels=len(LABEL2ID),
    id2label=ID2LABEL,
    label2id=LABEL2ID
)
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=-1)
    true_labels = [[ID2LABEL[l] for l in lab_seq if l != -100] for lab_seq in labels]
    true_preds = [[ID2LABEL[p_] for p_, l in zip(pred, lab_seq) if l != -100]
                   for pred, lab_seq in zip(preds, labels)]
    res = metric.compute(predictions=true_preds, references=true_labels)
    return {"precision": res["overall_precision"], "recall": res["overall_recall"], "f1": res["overall_f1"]}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}  ({100 * trainable_params/total_params:.2f}%)")

Total parameters: 332,505,165
Trainable parameters: 332,505,165  (100.00%)


In [ ]:
training_args = TrainingArguments(
    output_dir = "/content/models/michiyasunaga/BioLinkBERT-large",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    gradient_accumulation_steps=2,
    fp16=True,
    weight_decay=0.01,
    push_to_hub=False,
    logging_strategy="epoch",
    logging_dir="/content/models/michiyasunaga/BioLinkBERT-large/logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.747700,0.042194,0.964005,0.941830,0.952789
2,0.052800,0.031172,0.967906,0.956901,0.962372
3,0.031500,0.024651,0.974274,0.966288,0.970264
4,0.023700,0.021437,0.971142,0.974352,0.972745
5,0.015900,0.020593,0.974014,0.976203,0.975107
6,0.009400,0.019072,0.978376,0.980962,0.979667
7,0.006400,0.020368,0.979923,0.980830,0.980377
8,0.004900,0.019832,0.980077,0.982020,0.981047
9,0.003700,0.021493,0.979334,0.983607,0.981466
10,0.003000,0.021696,0.980359,0.983210,0.981782


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1380, training_loss=0.08991012713615446, metrics={'train_runtime': 432.6183, 'train_samples_per_second': 204.014, 'train_steps_per_second': 3.19, 'total_flos': 2.1287138743642176e+16, 'train_loss': 0.08991012713615446, 'epoch': 10.0})

In [ ]:
save_directory = "/content/drive/MyDrive/aicup/models/michiyasunaga/BioLinkBERT-large"
os.makedirs(save_directory, exist_ok=True)
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Model saved to {save_directory}")

Model saved to /content/drive/MyDrive/aicup/models/michiyasunaga/BioLinkBERT-large


# Output result-Token-Classification+Regex+延伸標籤

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json, re, os
from collections import defaultdict
import pandas as pd
import torch
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForTokenClassification, pipeline
)

# ── 0. 路徑設定 ─────────────────────────────────────────
ADAPTER    = "/content/drive/MyDrive/aicup/models/michiyasunaga/BioLinkBERT-large/2nd"
WS_JSON    = "/content/drive/MyDrive/task1_word_timestamps.json"
VALID_TXT  = "/content/drive/MyDrive/task1_answer.txt"
OUT_TXT    = "/content/drive/MyDrive/submission/task2_answer.txt"

BATCH_SIZE  = 16
CHUNK_SIZE  = 500
OVERLAP     = 50

# ── 1. 分數閾值 ───────────────────────────────────────
LABEL_THR = defaultdict(lambda: 0.65)
LABEL_THR.update({
    "DATE":0.70, "ZIP":0.70,
    "PATIENT":0.70, "PERSONALNAME":0.0,
    "CITY":0.70, "COUNTRY":0.75,
    "PROFESSION":0.0, "STATE":0.70,
    "ID_NUMBER":0.0, "MEDICAL_RECORD_NUMBER":0.0,
    "TIME":0.45, "COUNTY":0.45,
    "FAMILYNAME":0.85, "LOCATION-OTHER":0.95,
    "AGE":0.0
})

# ── 2. token 正則 & 工具函式 ──────────────────────────
WORD_RE = re.compile(
    r"[A-Za-z]+(?:'[A-Za-z]+)?|"   # 英文字 (含 don't)
    r"\d+(?:\.\d+)?|"        # 數字，可带 .AB
    r"[\u4e00-\u9fff]"        # 單一中文字
)
PUNC = set(".,:;!?，。；！？")
digits_only = lambda s: re.sub(r"\D", "", s)

def norm(w: str):
    w = w.lower()
    while w and w[0] in PUNC: w = w[1:]
    while w and w[-1] in PUNC: w = w[:-1]
    return w

# ── 3. 讀取 word-level timestamps ───────────────────────
def load_word_ts(path: str):
    with open(path, encoding="utf8") as f:
        raw = json.load(f)
    out = {}
    for fid, info in raw.items():
        if isinstance(info, list):
            lst = info
        else:
            lst = sum((seg.get("words", []) for seg in info["segments"]), [])
        out[fid] = [
            {"word": norm(w["word"]), "start": w["start"], "end": w["end"]}
            for w in lst if norm(w["word"])
        ]
    return out

word_ts = load_word_ts(WS_JSON)

# ── 4. 模型 pipeline ──────────────────────────────────
tok = AutoTokenizer.from_pretrained(ADAPTER, trust_remote_code=True)
tok.padding_side = "left"; tok.model_max_length = 512
cfg = AutoConfig.from_pretrained(ADAPTER, trust_remote_code=True)
model = (AutoModelForTokenClassification
         .from_pretrained(ADAPTER, config=cfg,
                          torch_dtype=torch.float16,
                          trust_remote_code=True)
         .eval().cuda())
ner_pipe = pipeline("token-classification",
                    model=model, tokenizer=tok,
                    device=0, aggregation_strategy="simple",
                    batch_size=BATCH_SIZE)

# ── 5. ext → base label map ────────────────────────────
ext2base = {
    "PATIENT_NAME":"PATIENT","SUBJECT":"PATIENT",
    "DOCTOR_NAME":"DOCTOR","PHYSICIAN":"DOCTOR","DR":"DOCTOR",
    "LOGIN_NAME":"USERNAME","USER_ID":"USERNAME",
    "EMAIL_ADDRESS":"EMAIL","MAIL":"EMAIL",
    "IP_ADDR":"IPADDRESS","IP_ADDRESS":"IPADDRESS",
    "SSN":"SOCIAL_SECURITY_NUMBER","MRN":"MEDICAL_RECORD_NUMBER",
    "HPN":"HEALTH_PLAN_NUMBER","ACCT_NO":"ACCOUNT_NUMBER","LIC_NO":"LICENSE_NUMBER",
    "VIN":"VEHICLE_ID","DEV_ID":"DEVICE_ID",
    "FINGERPRINT_ID":"BIOMETRIC_ID","FACE_ID":"BIOMETRIC_ID",
    "ID_NO":"ID_NUMBER","PASSPORT_NUMBER":"ID_NUMBER",
}

# ── 6. 高精度 regex ─────────────
rgx = {
    "TIME": re.compile(
        r"\b(?:[01]?\d|2[0-3]):[0-5]\d\b"
        r"|\b(?:morning|evening|tonight|midnight|noon|"
        r"last night|this evening|nighttime|afternoon)\b",
        flags=re.IGNORECASE
    ),
    "MEDICAL_RECORD_NUMBER": re.compile(
        r"\b\d{5,9}(?:\.[A-Z]{2,4})?\b", flags=re.IGNORECASE
    ),
}

# ── 7. chunk helper ───────────────────────────────────
def chunk_text(text: str):
    out, s = [], 0
    while s < len(text):
        e = min(s + CHUNK_SIZE, len(text))
        out.append(text[s:e])
        s = e - OVERLAP if e < len(text) else e
    return out

# ── 8. align_tokens：对 MRN／ID_NUMBER 用不同備援 ─────
num_pat = re.compile(r"^[0-9\-\:\s]+$")

def align_tokens(fid: str, raw_norm: str, tokens, label: str):
    """
    raw_norm: norm(raw) 的結果，例如 "3:30pm"、"07:15"、"1234567.ab"
    tokens: WORD_RE.findall(raw) 的 token list（normalize 後），例如 ["3","30","pm"]
    label: 字串標籤，例如 "ID_NUMBER", "MEDICAL_RECORD_NUMBER", "TIME" 等
    """
    # 1. 把 tokens normalize（去 punctuation、to lower）之後，濾掉空字串
    tokens_norm = [norm(t) for t in tokens if norm(t)]
    # 2. 從 word_ts 取出對應的 normalized word 序列
    words = [w["word"] for w in word_ts.get(fid, [])]
    n, m = len(words), len(tokens_norm)
    if m == 0 or n < m:
        return None, None

    # --- Step 0：如果 raw_norm 直接在 words 裡，就可以直接命中 ---
    if raw_norm in words:
        idx = words.index(raw_norm)
        return word_ts[fid][idx]["start"], word_ts[fid][idx]["end"]

    # --- Step 1：嚴格連續匹配 （words[i:i+m] == tokens_norm） ---
    for i in range(n - m + 1):
        if words[i:i+m] == tokens_norm:
            return word_ts[fid][i]["start"], word_ts[fid][i+m-1]["end"]

    # --- Step 2：原本對 ID_NUMBER／MRN 之類的 fallback ---
    dig_tokens = [digits_only(t) for t in tokens_norm]
    if label == "ID_NUMBER":
        # ID_NUMBER: 只要每個 token 經 digits_only 後都還有內容，就將它們都串成 one sequence 來對
        if all(d for d in dig_tokens):
            seq = [digits_only(w) for w in words]
            for i in range(n - m + 1):
                if seq[i:i+m] == dig_tokens:
                    return word_ts[fid][i]["start"], word_ts[fid][i+m-1]["end"]
    else:
        # MRN & 其他標籤: 只有 tokens_norm 全部符合 num_pat （純數字/冒號/連字符/空格）才做這一步
        if all(num_pat.fullmatch(t) for t in tokens_norm):
            seq = [digits_only(w) for w in words]
            tgt = [digits_only(t) for t in tokens_norm]
            for i in range(n - m + 1):
                if seq[i:i+m] == tgt:
                    return word_ts[fid][i]["start"], word_ts[fid][i+m-1]["end"]

    # --- Step 3：對於 TIME 類別，先嘗試「只比對連續的 digit-only 序列」---
    if label == "TIME":
        # 把 tokens_norm 裡面每個 token 做 digits_only：
        # e.g. tokens_norm = ["3","30","pm"] -> dig_only = ["3","30"]
        dig_only = [d for d in (digits_only(t) for t in tokens_norm) if d]
        if dig_only:
            seq = [digits_only(w) for w in words]
            k = len(dig_only)
            for i in range(n - k + 1):
                if seq[i:i+k] == dig_only:
                    return word_ts[fid][i]["start"], word_ts[fid][i+k-1]["end"]

    # --- Step 4：再加一層「只要任一子 token 或其數字版能對上，就收集所有 match 的 timestamp」---
    # 目標：只要 tokens_norm 裡面任一個 (a) 直接在 words 裡，或 (b) digits_only 版本在 words 的 digits_only 序列裡，
    # 就都把那些匹配上的 word_ts start/end 收集起來，最後以 min(start) 當起點，max(end) 當終點。
    if label == "TIME":
        matched_starts = []
        matched_ends = []

        # (1) 建立一個方便查 digits_only(words[j]) 的 list
        seq_digits = [digits_only(w) for w in words]

        for t in tokens_norm:
            # a. 先去找「words[j] == t」的所有位置
            for j, w in enumerate(words):
                if w == t:
                    matched_starts.append(word_ts[fid][j]["start"])
                    matched_ends.append(word_ts[fid][j]["end"])
            # b. 再去找「digits_only(w) == digits_only(t) != ''」的所有位置
            t_dig = digits_only(t)
            if t_dig:
                for j, wdig in enumerate(seq_digits):
                    if wdig == t_dig:
                        matched_starts.append(word_ts[fid][j]["start"])
                        matched_ends.append(word_ts[fid][j]["end"])

        if matched_starts and matched_ends:
            return min(matched_starts), max(matched_ends)

    # 都找不到，就回 None
    return None, None

# ── 9. predict_one （主推論） ──────────────────────────
def predict_one(fid: str, text: str):
    used = set()
    results = []

    # A. model 識別（與原本一樣）
    for seg in chunk_text(text):
        for ent in ner_pipe(seg):
            lbl = ext2base.get(ent["entity_group"].upper(),
                               ent["entity_group"].upper())
            if ent["score"] < LABEL_THR[lbl]:
                continue
            raw = ent["word"]
            raw_norm = norm(raw)
            if not raw_norm or (lbl, raw_norm) in used:
                continue

            toks = WORD_RE.findall(raw)
            s, e = align_tokens(fid, raw_norm, toks, lbl)
            if s is None:
                continue
            results.append((fid, lbl, s, e, raw_norm))
            used.add((lbl, raw_norm))

    # B. regex 識別（TIME / MRN，原本那段不動）
    lower = text.lower()
    for lbl, ptn in rgx.items():
        for m in ptn.finditer(lower):
            raw = m.group(0)
            raw_norm = norm(raw)
            if (lbl, raw_norm) in used:
                continue
            toks = WORD_RE.findall(raw)
            s, e = align_tokens(fid, raw_norm, toks, lbl)
            if s is None:
                continue
            results.append((fid, lbl, s, e, raw_norm))
            used.add((lbl, raw_norm))

    # B2. 額外補上 "H(或HH)am"/"H(或HH)pm" 等常見格式
    for m2 in re.finditer(r"\b(\d{1,2})(?::(\d{2}))?\s*(am|pm)\b", lower):
        raw = m2.group(0)
        raw_norm = norm(raw)
        lbl = "TIME"
        if (lbl, raw_norm) in used:
            continue
        toks = WORD_RE.findall(raw)
        s, e = align_tokens(fid, raw_norm, toks, lbl)
        if s is None:
            # 若對不上，再做一次「只比對純數字序列」fallback
            digits_seq = [digits_only(t) for t in toks if digits_only(t)]
            if digits_seq:
                words_seq = [digits_only(w["word"]) for w in word_ts.get(fid, [])]
                k = len(digits_seq)
                for i in range(len(words_seq) - k + 1):
                    if words_seq[i:i+k] == digits_seq:
                        s = word_ts[fid][i]["start"]
                        e = word_ts[fid][i+k-1]["end"]
                        break
        if s is not None:
            results.append((fid, lbl, s, e, raw_norm))
            used.add((lbl, raw_norm))

    # C. Δt<0.10s 同 label 去重（與原本一樣）
    results.sort(key=lambda r: (r[1], r[2], r[3]))
    final = []
    for r in results:
        if any(r[1] == f[1] and abs(r[2] - f[2]) < 0.10 for f in final):
            continue
        final.append(r)
    return final

# ── 10. 批次跑所有檔案 ─────────────────────────────────
df = pd.read_csv(VALID_TXT, sep="\t",
                 names=["fid", "content"], dtype=str, header=None)

rows = []
for i in range(0, len(df), BATCH_SIZE):
    for rec in df.iloc[i:i+BATCH_SIZE].itertuples(index=False):
        rows.extend(predict_one(rec.fid, rec.content))

with open(OUT_TXT, "w", encoding="utf8") as f:
    for fid, lbl, s, e, raw_norm in rows:
        f.write(f"{fid}\t{lbl}\t{s:.3f}\t{e:.3f}\t{raw_norm}\n")

print("Done. lines:", len(rows))


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Done. lines: 2532
